In [ ]:
#1
# объявление основных переменных/функций


import os
import os.path
from google.colab import output as cou
from google.colab import drive
import re
from re import search
import subprocess
from subprocess import getoutput
import threading
from IPython.display import HTML, Javascript, display, clear_output
from ipython_genutils.py3compat import builtin_mod
import ipywidgets as widgets
import requests
import glob
import time
from datetime import timedelta
import distutils.dir_util
from IPython.utils import capture
import multiprocessing
import random
import string
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle


cfg = "/content/.config/"
#переменная !$dl для сокращения команды загрузки, перед использованием обязательно задать переменную пути $path
path = '/content/'
wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "
dl = f"{wget} -P {path} "
sf = "https://huggingface.co/utnah/safetensors/resolve/main/"
pt = "https://huggingface.co/utnah/ckpt/resolve/main/"
esrgan = "https://huggingface.co/utnah/esrgan/resolve/main/"
lora = "https://huggingface.co/utnah/lora/resolve/main/"
hyper = "https://huggingface.co/utnah/hypernetworks/resolve/main/"
dl_deps = "wget -q -t 10 --content-disposition https://huggingface.co/utnah/deps/resolve/main/"
deps = "https://huggingface.co/utnah/deps/resolve/main/"
gh = "https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/"
dl_gh = f"wget -q -t 10 --content-disposition  {gh}"
end = '/content/.config/end'
deps_bar=f"{gh}depsprogressbar" # прогрессбар во время распаковки пакетов: loader(deps)
inprogress=f"{gh}loader_anim" # заглушка на css во время выполнения чего-либо: loader(inprogress)
import requests
m = HTML(requests.get(gh+'st').text)
ok = ""
with capture.capture_output() as stored_variables:
  %store cfg
  %store wget
  %store sf
  %store pt
  %store esrgan
  %store lora
  %store hyper
  %store dl_deps
  %store deps
  %store gh
  %store dl_gh
  %store end
  %store deps_bar
  %store inprogress
  %store m

def loader(html): # для подгрузки html/svg
  display(HTML(html, metadata=dict(isolated=True)))  

#простая качалка (для загрузки архива с зависимостями)
def download(url: str, fname: str, desc: str, chunk_size=1024):
  try:
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
      desc=desc,
      total=total,
      unit='iB',
      unit_scale=True,
      unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
          size = file.write(data)
          bar.update(size)
  except KeyboardInterrupt:
    print("\nзагрузка отменена")
  except Exception as error:
    print(f"загрузка не удалась:\n{error}")

def ng():
  !$dl_gh'end' -O /content/.config/end # svg-баннер после завершения
  !$dl_gh'zrpmdNELTAkW' -O /content/.config/cfg # нгрок-токены

def depinst(): #распаковка зависимостей (которые ставятся через pip)
  time.sleep(1.5)
  #!$dl_deps'd'
  download(f"{deps}d", "/content/d", "зависимости")
  clear()
  loader(deps_bar)
  !7z -bso0 -bd -mmt4 -slp -y x /content/d -o/ 2>/dev/null
  !rm /content/d

def flara(): # запрос в клаудфлару, пока не выдаст ссылку
  import os
  from subprocess import getoutput
  while True:
    !cloudflared tunnel --url localhost:7860 > clf.txt 2>&1 &
    !sleep 9
    !grep -m2 -o 'https[^ ]*' /content/stable-diffusion-webui/clf.txt | tail -n1 > clfr.txt
    srvs = getoutput('cat /content/stable-diffusion-webui/clf.txt')
    if "has been created" in srvs:
      break
    !rm /content/clf.txt
    !rm /content/clfr.txt
  cflare = getoutput('cat /content/stable-diffusion-webui/clfr.txt')

#проигрывание аудио, чтобы вкладка не убивалась
def mobile_player():
  cou.eval_js("var audio = new Audio('https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/psyattack.wav'); audio.addEventListener('ended', function() { audio.currentTime = 0; audio.play(); }, false); audio.play();")

# css-файл для внедрения в виджеты
css_setup_widget = requests.get(f"{gh}css_setup_widget.html").text
def setup_css():
  display(HTML(f'''
    {css_setup_widget}
'''))
  

# установка времени запуска колаба для таймера
try:
  launching
except:
  launching = int(time.time())-6
  with capture.capture_output() as stored_variables:
    %store launching